In [38]:
import os
os.environ['KERAS_BACKEND'] = 'theano'

import pandas as pd
import numpy as np

import keras
from keras.preprocessing.text import Tokenizer
from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback
from keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed
from keras import backend as K
from keras.models import Model

from sklearn.metrics import roc_auc_score

import input as dataset
import tensorflow as tf
from network import Chemception
from network import SMILE2Vector 
from network import HATT
import input as data
import helpers
import Optimizer
from evaluation import Metrics

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.callbacks import TensorBoard
import keras.backend as K

import os
import sys
import time
import statistics
import shutil

import numpy as nu

from sklearn.model_selection import train_test_split



from string import punctuation
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [39]:
class AttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(AttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init((self.attention_dim, )))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.trainable_weights = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])


In [40]:
MAX_WORD_LENGTH = 180
MAX_NB_CHARS = 180
EMBEDDING_DIM = 180

In [41]:
dataComp = dataset.LoadData('data',0)
smiles = list(map(lambda x: x._SMILE, dataComp))

Loading Started
Load of 24003 finished in 5.768050909042358s


In [42]:
tokenizer = Tokenizer(num_words=None, char_level=True)
tokenizer.fit_on_texts(smiles)
print(smiles[0])

O=C2N(Cc1ccccc1)CC(=O)C23(CC3)


In [43]:
dictionary = {}
i=0
k=0
for smile in smiles:
    i+=1
    for c in list(smile):
        k+=1
        if c in dictionary:
            dictionary[c]+=1
        else:
            dictionary[c]=1
print(len(dictionary))

53


In [44]:
# sequence encode
encoded_docs = tokenizer.texts_to_sequences(smiles)

# pad sequences
max_length = max([len(s) for s in smiles])
Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

# define vocabulary size (largest integer value)
vocab_size = len(tokenizer.word_index) + 1


In [45]:
print(max_length)

498


In [46]:
labels = list(map(lambda x: 1 if x.mutagen==True else 0,dataComp))

In [47]:
X = Xtrain
Y = keras.utils.np_utils.to_categorical(labels, 2)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=round(0.2*len(encoded_docs)), random_state=7)


In [48]:
print(X_train[1:10])
print(len(X_train[0]))

[[1 2 7 ... 0 0 0]
 [5 6 1 ... 0 0 0]
 [5 6 1 ... 0 0 0]
 ...
 [5 1 7 ... 0 0 0]
 [1 1 3 ... 0 0 0]
 [1 7 1 ... 0 0 0]]
498


In [49]:
model = Sequential()
model.add(Embedding(vocab_size+1, 100, input_length=max_length,
                    trainable = True,
                    mask_zero=False))
model.add(Bidirectional(GRU(100, return_sequences=True)))
model.add(AttLayer(100))
model.add(Dense(2, activation='softmax'))


In [50]:
model.compile(loss='mean_squared_error',
              		optimizer='rmsprop',
              		metrics=['acc'])
model.fit(X_train, 
           Y_train, validation_data=(X_test, Y_test),epochs=10)

Train on 19202 samples, validate on 4801 samples
Epoch 1/10
19202/19202 [==============================] - 449s 23ms/step - loss: 0.1784 - acc: 0.7445 - val_loss: 0.1721 - val_acc: 0.7498
Epoch 2/10
19202/19202 [==============================] - 451s 23ms/step - loss: 0.1665 - acc: 0.7643 - val_loss: 0.1914 - val_acc: 0.7069
Epoch 3/10
 4896/19202 [======>.......................] - ETA: 7:26 - loss: 0.1630 - acc: 0.7663

KeyboardInterrupt: 

In [ ]:
print(Y_test.shape)